In [ ]:
import spotipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import io
import cv2
import os
from PIL import Image
import pathlib
import csv
import warnings
warnings.filterwarnings('ignore')

In [ ]:
url_dataset = 'https://github.com/Ferdinand-Genans/SpotifyProject/blob/main/dataset.csv?raw=true'
df = pd.read_csv(url_dataset, index_col=0) #telechargement du dataset des musiques Spotify
df = df.dropna()
genres = ['classical','country', 'house', 'metal', 'k-pop', 'jazz', 'rock', 'pop', 'r-n-b', 'reggae']
df = df.loc[df.track_genre.isin(genres)]
df = df.reset_index() #pour que les indices partent de 0

In [ ]:
def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=dpi)
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def crop_white(array):
    image = array
    # Selectionne tous les pixels qui sont pratiqquement b
    white = np.array([255, 255, 255])
    mask = np.abs(image - white).sum(axis=2) < 10

    # Find the bounding box of those pixels
    coords = np.array(np.nonzero(~mask))
    top_left = np.min(coords, axis=1)
    bottom_right = np.max(coords, axis=1)

    out = image[top_left[0]:bottom_right[0],
                top_left[1]:bottom_right[1]]
    return out

def preprocessing_audio_chroma(path_in, path_out):
    y, sr = librosa.load(path_in)
    length = len(y)//3
    for i in range(3):
        path_out_ = path_out+"_chroma_"+str(i)+".png"
        y_ = y[i*length:(i+1)*length]
        C = librosa.feature.chroma_cqt(y=y_, sr=sr)
        fig, ax = plt.subplots(figsize=(2.5, 2.5))
        img = librosa.display.specshow(C, ax=ax, cmap="gray")
        array = get_img_from_fig(fig)
        array_cropped = crop_white(array)
        im = Image.fromarray(array_cropped)
        print("saving", path_out_)
        im.save(path_out_)

In [ ]:
genres_ = ['classical','country', 'house', 'metal', 'k-pop', 'jazz', 'rock']

for g in genres:
    path = g+"/"
    name_chroma = g+"_chroma"
    os.mkdir(name_chroma)
    name_cqt = g+"_cqt"
    os.mkdir(name_cqt)
    for root, dirs, files in os.walk(g):
        for file in files:
            name = file.split(".")[0]
            path_in = path+file
            path_out_chroma = name_chroma+"/"+name
            try: 
                preprocessing_audio_chroma(path_in, path_out_chroma)
            except: 
                print("Error:", name)         